Understanding Factual Accuracy & Hallucination Detection
Large Language Models (LLMs) sometimes "hallucinate"—meaning they generate incorrect or misleading information. To evaluate and mitigate this, we use different fact-checking methods:

### TruthfulQA

A benchmark designed to measure how well LLMs avoid falsehoods across multiple categories. It contains 817 questions designed to test common misconceptions, false beliefs, and factual errors. Models are evaluated based on:

- **Truthfulness**: Does the answer align with verified knowledge?
- **Informative Score**: Is the answer relevant and detailed?

### Attributable to Source (ATS)

Measures whether the generated text aligns with a given source (retrieved documents, datasets, etc.). Commonly used in Retrieval-Augmented Generation (RAG) pipelines. Example metric: How much of the model's response can be traced back to retrieved sources?

### RAG-based Fact Checking

Uses external retrieval models (e.g., FAISS, ChromaDB) to verify if model-generated outputs align with real-world knowledge. The process involves:

1. The model generates an answer.
2. A retrieval system finds relevant documents.
3. The response is compared to the retrieved facts.

In [3]:
!pip install faiss-cpu chromadb langchain

In [4]:
from llama_cpp import Llama

# Load the model (download a GGUF file, e.g., Mistral-7B)
llm = Llama(model_path="/Users/harshbhatt/Projects/ai-projects/book-reader/gguf/mistral-7b-instruct-v0.1.Q4_K_M.gguf", n_gpu_layers=40)

# Generate a response
def generate_response(prompt):
    output = llm(prompt, max_tokens=200)
    return output["choices"][0]["text"].strip()

# Test the model
print(generate_response("Who developed the theory of relativity?"))

llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 12287 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/harshbhatt/Projects/ai-projects/book-reader/gguf/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_

Albert Einstein


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load a local embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Lightweight & efficient

# Sample documents (expand with real data)
documents = [
    "The capital of France is Paris.",
    "Einstein developed the theory of relativity.",
    "The mitochondrion is the powerhouse of the cell."
]

# Compute embeddings
doc_vectors = np.array(embedder.encode(documents), dtype=np.float32)

# Build FAISS index
dimension = doc_vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_vectors)

print("Knowledge base indexed!")


/Users/harshbhatt/miniconda3/envs/qunta-backend/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

Knowledge base indexed!


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def fact_check(query, similarity_threshold=0.7):
    # Generate response using the local LLM
    response = generate_response(query)
    
    # Embed both response and retrieved fact
    response_vector = np.array(embedder.encode([response]), dtype=np.float32)
    _, nearest_ids = index.search(response_vector, k=1)
    retrieved_fact = documents[nearest_ids[0][0]]
    retrieved_vector = np.array(embedder.encode([retrieved_fact]), dtype=np.float32)

    # Compute similarity
    similarity = cosine_similarity(response_vector, retrieved_vector)[0][0]

    # Print results
    print(f"\n🔹 Question: {query}")
    print(f"🤖 LLM Response: {response}")
    print(f"📚 Retrieved Fact: {retrieved_fact}")
    print(f"📊 Cosine Similarity: {similarity:.2f}")

    # Check factual accuracy based on similarity score
    if similarity >= similarity_threshold:
        print("✅ Response is factually accurate.")
    else:
        print("❌ Possible hallucination detected!")

# Test
fact_check("Who developed the theory of relativity?")


Llama.generate: 8 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    2351.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    2905.07 ms /    55 runs   (   52.82 ms per token,    18.93 tokens per second)
llama_perf_context_print:       total time =    2919.30 ms /    56 tokens



🔹 Question: Who developed the theory of relativity?
🤖 LLM Response: The theory of relativity was developed by Albert Einstein. It consists of two parts: the Special Theory of Relativity, which was published in 1905, and the General Theory of Relativity, which was published in 1915.
📚 Retrieved Fact: Einstein developed the theory of relativity.
📊 Cosine Similarity: 0.83
✅ Response is factually accurate.
